# Day 19

https://adventofcode.com/2020/day/19

## Part 1

I made several tries, but ultimately I reckon I cannot solve it withour using regexp. so I need to learn the basics...

In [404]:
# Here's how I can use a rule as a regexp, using the OR sign and parentheses...
import re
bool(re.fullmatch('a(b|a)','aa'))

True

In [401]:
#filename = "data/day19test1.txt"
filename = "data/input19.txt"

with open(filename) as f:
    p = [b.split("\n") for b in f.read().split("\n\n")]

messages = p[1][:-1]

# store rules in dictionary with rule number as key (as a string)
rules = {}
for l in p[0]:
    i, r = l.split(": ")
    rules[i] = r.replace('"','')

# add parentesys (and space to that later split will work) if there's an OR symbol in rule.
for r in rules.keys():
    if "|" in rules[r]:
        rules[r] = "( "+rules[r]+" )"

# dictionay of solved rules
rules_solved = {}

while '0' not in rules_solved.keys():

    for k in rules.keys():
        if k in rules_solved.keys():
            continue

        r = rules[k]        
        if not re.findall("[0-9]",rules[k]): # rule is solved, store it
            rules_solved[k] = r
        else: # otherwise see whether I can replace rule number with corresponding rule
            l = r.split(" ")
            for i in range(len(l)):
                j = l[i]
                if j in rules_solved.keys():
                    #r = r.replace(j,rules_solved[j]) ## HERE'S THE BUG! there could be a number containig the rule
                                                      ## number, e.g. 133 contains 13, replace will screw the string
                    l[i] = rules_solved[j] # solve by replacing item in list, and the reforming string "by hand"..
            nr = "" # reform rule string after replacements
            for j in l:
                nr += j+" "
            nr = nr[:-1] # remove last white space
            rules[k] = nr
    
rule0 = rules_solved['0'].replace(" ","") # remove all white spaces from rule 0
print(rule0)

(b((a(a(b(a(ba|bb)|b(ba|ab))|a((aa|bb)a|aab))|b(a(aa|ba)(b|a)|b(a(bb|a(b|a))|baa)))|b(a(a(b(ba|bb)|a(aa|ba))|b(aab|aba))|b(b(b(aa|ba)|a(ba|a(b|a)))|a(ab|bb)a)))b|((b((abb|(ba|ab)a)a|((ba|(b|a)b)b|(ba|a(b|a))a)b)|a(b(b(ba|a(b|a))|aaa)|a(b|a)(ab|bb)))a|(a(a(a(b(b|a)|aa)|b(ab|bb))|b((b(b|a)|aa)b|(ba|(b|a)b)a))|b(b(b|a)(bb|a(b|a))|a(b(ba|ab)|a(ba|(b|a)b))))b)a)|a((b((b(a(ba|(b|a)b)|b(bb|a(b|a)))|a(aaa|(aa|bb)b))a|((b(ab|bb)|a(bb|a(b|a)))a|(b(ba|bb)|aab)b)b)|a(b(a(ab|bb)b|b(a(ba|(b|a)b)|baa))|a(((ba|a(b|a))b|aba)b|(b(ba|bb)|a(b(b|a)|aa))a)))a|(((b(aab|(aa|ba)a)|a(ab|bb)b)a|(b(ba|ab)b|(b|a)(ba|(b|a)b)a)b)b|(a((abb|(ba|a(b|a))a)b|((aa|bb)a|(aa|ba)b)a)|b(((b(b|a)|aa)b|aba)a|(baa|a(ba|a(b|a)))b))a)b))(b((a(a(b(a(ba|bb)|b(ba|ab))|a((aa|bb)a|aab))|b(a(aa|ba)(b|a)|b(a(bb|a(b|a))|baa)))|b(a(a(b(ba|bb)|a(aa|ba))|b(aab|aba))|b(b(b(aa|ba)|a(ba|a(b|a)))|a(ab|bb)a)))b|((b((abb|(ba|ab)a)a|((ba|(b|a)b)b|(ba|a(b|a))a)b)|a(b(b(ba|a(b|a))|aaa)|a(b|a)(ab|bb)))a|(a(a(a(b(b|a)|aa)|b(ab|bb))|b((b(b|a)|aa)b|(ba|(

In [402]:
import re

match = 0
for m in messages:
    #print(m,bool(re.fullmatch(rule0,m)))
    if bool(re.fullmatch(rule0,m)):
        match += 1
        
print('Solution Part 1 =',match)

Solution Part 1 = 126
